In [3]:
import json
import requests
import yaml
import pandas as pd
import warnings
from bs4 import BeautifulSoup
import re
import numpy as np
from shapely.geometry import Point
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# key_dict = {'key' : ''}

In [5]:
# dump the key into yaml 
with open('credentials.yml','w') as output:
    yaml.dump(key_dict, output)
    
# get the key from .yaml file
key_dict = yaml.load(open('credentials.yml'), Loader=yaml.FullLoader)  

In [6]:
key_dict = yaml.load(open("credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = key_dict["key"]
api_key = gmaps_key

### Scraping Coop locations in Zürich

In [7]:
page_Lidl = requests.get("https://www.profital.ch/de/flr/72148-lidl-filialen/2699")
print(page_Lidl.status_code) ## prints 200 as status code
#print(page.content[:3000]) ## prints the entire webpage content
soup_Lidl = BeautifulSoup(page_Lidl.content, "html.parser")
type(soup_Lidl)

200


bs4.BeautifulSoup

In [8]:
soup_Lidl.findAll('address');

In [9]:
Loc_Lidl=[]
Loc_Lidl.append(soup_Lidl.findAll('address'))
page_num_Lidl=len(Loc_Lidl)
page_num_Lidl

1

In [11]:
Address_Lidl=[]
Zip_Lidl=[]
for i in range(0,page_num_Lidl):
    for j in range(0,len(Loc_Lidl[i])):
        Address_Lidl.append(Loc_Lidl[i][j].contents[0].strip())
        Zip_Lidl.append(Loc_Lidl[i][j].contents[2].strip().split()[0])

In [13]:
df_Lidl_Locator = pd.DataFrame(list(zip(Address_Lidl,Zip_Lidl)), columns=["Address", "Zip_Code"])
df_Lidl_Locator

,Address,Zip_Code
0,Fraumünsterstrasse 16,8001
1,Förrlibuckstrasse 62,8005
2,Grubenstrasse 28,8045
3,Hofwiesenstrasse 350,8050
4,Ohmstrasse 11,8050
5,Wehntalerstrasse 625,8046
6,Überlandstrasse 381,8051


In [14]:
df_Lidl_Address = pd.DataFrame()
# Combine Address and Zip Code into a single column
df_Lidl_Address["address"] = df_Lidl_Locator["Address"] + ', ' + df_Lidl_Locator["Zip_Code"]
df_Lidl_Address

,address
0,"Fraumünsterstrasse 16, 8001"
1,"Förrlibuckstrasse 62, 8005"
2,"Grubenstrasse 28, 8045"
3,"Hofwiesenstrasse 350, 8050"
4,"Ohmstrasse 11, 8050"
5,"Wehntalerstrasse 625, 8046"
6,"Überlandstrasse 381, 8051"


### Getting store locations from Google Maps and mapping the location to Kreis name

In [15]:
from shapely.geometry import shape, GeometryCollection, Point

In [16]:
# Function to get latitude and longitude from Google Maps API
def get_lat_lng(api_key, query):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    req = requests.get(url + "query=" + query + "&key=" + api_key)
    places_json = req.json()
    my_result = places_json.get("results", [])

    if my_result:
        location = my_result[0].get("geometry", {}).get("location", {})
        lat, lng = location.get("lat"), location.get("lng")
        return lat, lng
    else:
        return None, None

In [17]:
# Function to get kreis_name from GeoJSON data
def get_kreis_name(point, geojson_data):
    for feature in geojson_data["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]["bezeichnung"]
    return None

In [19]:
# Load GeoJSON data
with open('data/stzh.adm_stadtkreise_a.json', 'r') as f:
    geojson_data = json.load(f)

# Create a DataFrame to store the results
columns = ["address", "latitude", "longitude", "kreis_name"]
lidl_df = pd.DataFrame(columns=columns)

# Iterate through addresses in the DataFrame
for index, row in df_Lidl_Address.iterrows():
    address = row["address"]
    
    # Get latitude and longitude from Google Maps API
    lat, lng = get_lat_lng(api_key, address)
    
    # If coordinates are found, create a Point object
    if lat is not None and lng is not None:
        point = Point(lng, lat)
        
        # Get kreis_name from GeoJSON data
        kreis_name = get_kreis_name(point, geojson_data)
        
        # Append results to DataFrame
        lidl_df = pd.concat([lidl_df, pd.DataFrame([{"address": address, "latitude": lat, "longitude": lng, "kreis_name": kreis_name}])], ignore_index=True)

# Display the final DataFrame
lidl_df

,address,latitude,longitude,kreis_name
0,"Fraumünsterstrasse 16, 8001",47.368829,8.541070,Kreis 1
1,"Förrlibuckstrasse 62, 8005",47.392027,8.514986,Kreis 5
2,"Grubenstrasse 28, 8045",47.361769,8.514720,Kreis 3
3,"Hofwiesenstrasse 350, 8050",47.410351,8.543040,Kreis 11
4,"Ohmstrasse 11, 8050",47.410340,8.545741,Kreis 11
5,"Wehntalerstrasse 625, 8046",47.422285,8.499196,Kreis 11
6,"Überlandstrasse 381, 8051",47.407813,8.585103,Kreis 12


In [20]:
lidl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   address     7 non-null      object 
 1   latitude    7 non-null      float64
 2   longitude   7 non-null      float64
 3   kreis_name  7 non-null      object 
dtypes: float64(2), object(2)
memory usage: 356.0+ bytes


In [21]:
# Save DataFrame to CSV
lidl_df.to_csv('lidl_data.csv', index=False)